# Manage multi-modal data

In [ ]:
!lamin init --storage ./test-multimodal --schema bionty

In [ ]:
import lamindb as ln
import lnschema_bionty as lb

lb.settings.species = "human"
ln.settings.verbosity = 3

In [ ]:
ln.track()

## MuData object

Let's use a MuData object:

In [ ]:
mdata = ln.dev.datasets.mudata_papalexi21_subset()

In [ ]:
mdata

First we register the file:

In [ ]:
file = ln.File(
    "papalexi21_subset.h5mu", description="Sub-sampled MuData from Papalexi21"
)
file.save()

## Register features

Now let's register the 3 feature sets this data contains:
1. rna
2. adt
3. obs (metadata)

### modalities

For the two modalities rna and adt, we use bionty tables as the reference:

In [ ]:
mdata["rna"].var_names[:5]

In [ ]:
feature_set_rna = ln.FeatureSet.from_values(
    mdata["rna"].var_names, field=lb.Gene.symbol
)

In [ ]:
mdata["adt"].var_names

In [ ]:
feature_set_adt = ln.FeatureSet.from_values(
    mdata["adt"].var_names, field=lb.CellMarker.name
)

Link them to file:

In [ ]:
file.features.add_feature_set(feature_set_rna, slot="rna")
file.features.add_feature_set(feature_set_adt, slot="adt")

### metadata

The 3rd feature set is the obs:

In [ ]:
obs = mdata["rna"].obs

In [ ]:
feature_set_obs = ln.FeatureSet.from_df(obs, "metadata")

In [ ]:
file.features.add_feature_set(feature_set_obs, slot="obs")

We'd also like to track a few metadata columns as labels:

In [ ]:
gene_targets = lb.Gene.from_values(obs["gene_target"], "symbol")
file.features.add_labels(gene_targets)

In [ ]:
labels = []
for col in ["orig.ident", "perturbation", "replicate", "Phase", "guide_ID"]:
    labels += ln.Label.from_values(obs[col])

In [ ]:
file.features.add_labels(labels)

In [ ]:
file.features

In [ ]:
file.describe()